In [1]:
from functions_test_toy_data import test_toy_data, extract_parents_pcmci, extract_parents_pcmciplus, extract_parents_lpcmci

In [2]:
test_toy_data(1, extract_parents_pcmci)

Predicted parents: {0: [(0, -1), (10, -1), (3, -3), (16, -1)], 1: [(16, -3), (10, -1), (1, -1), (4, -3), (17, -2), (2, -1)], 2: [(2, -1), (0, -2), (10, -1), (16, -2)], 3: [(7, -2), (15, -3), (14, -3), (5, -2), (1, -3)], 4: [(8, -1), (4, -1), (8, -2), (15, -3), (14, -2), (7, -1), (3, -1), (13, -1)], 5: [(5, -1), (5, -3), (11, -2), (14, -3), (9, -3), (18, -1), (14, -2)], 6: [(6, -1), (19, -2), (18, -3), (15, -2), (18, -1), (2, -2), (15, -3)], 7: [(7, -1), (8, -3), (10, -1), (13, -2)], 8: [(8, -1), (18, -1), (12, -1)], 9: [(15, -1), (13, -3), (14, -3), (9, -1), (7, -3)], 10: [(10, -1), (5, -1), (4, -1), (13, -1)], 11: [(11, -1), (13, -1), (19, -2), (13, -2)], 12: [(18, -3), (6, -2), (13, -2), (0, -3)], 13: [(13, -1), (9, -3), (14, -1)], 14: [(14, -1), (0, -1), (12, -2)], 15: [(15, -1), (10, -2), (13, -1), (9, -2)], 16: [(16, -1), (9, -2), (0, -1), (10, -3), (17, -1), (7, -2), (17, -2)], 17: [(17, -1), (19, -3), (5, -3), (18, -1), (6, -1), (18, -3)], 18: [(18, -1), (7, -3), (14, -2), (2, -

In [3]:
test_toy_data(1, extract_parents_pcmciplus)

Predicted parents: {0: [(0, -1)], 1: [(1, -1), (7, -3), (10, -1)], 2: [(2, -1)], 3: [(3, -1), (7, -2)], 4: [(8, -1), (4, -1), (8, -2)], 5: [(5, -1)], 6: [(6, -1), (7, -1), (18, -1)], 7: [(7, -1)], 8: [(8, -1), (18, -1)], 9: [(9, -1)], 10: [(10, -1), (1, -3), (8, -1)], 11: [(11, -1), (13, -1), (17, -1)], 12: [(6, -2)], 13: [(13, -1)], 14: [(14, -1), (0, -1)], 15: [(15, -1)], 16: [(16, -1)], 17: [(17, -1), (19, -3), (18, -3)], 18: [(18, -1), (14, -2)], 19: [(19, -1)]}
Causal process: {0: [(0, -1)], 1: [(1, -1)], 2: [(2, -1)], 3: [(3, -1)], 4: [(4, -1), (2, -3)], 5: [(5, -1)], 6: [(6, -1), (19, -2)], 7: [(7, -1)], 8: [(8, -1), (2, -2)], 9: [(9, -1)], 10: [(10, -1)], 11: [(11, -1)], 12: [(12, -1)], 13: [(13, -1)], 14: [(14, -1), (0, -3)], 15: [(15, -1)], 16: [(16, -1)], 17: [(17, -1), (18, -3)], 18: [(18, -1)], 19: [(19, -1)]}
Precision: 0.8
Recall: 0.5555555555555556
F1: 0.6557377049180328


In [4]:
test_toy_data(1, extract_parents_lpcmci)

Predicted parents: {0: [(0, -1), (3, -3), (10, -1), (18, -3)], 1: [(1, -1), (7, -3), (10, -1)], 2: [(2, -1)], 3: [(3, -1)], 4: [(4, -1), (8, -1), (8, -2), (15, -3)], 5: [(5, -1)], 6: [(6, -1), (7, -1), (19, -2)], 7: [(7, -1)], 8: [(8, -1), (18, -1)], 9: [(9, -1), (15, -1)], 10: [(10, -1), (13, -1)], 11: [(11, -1), (13, -1), (17, -1)], 12: [(4, -3), (12, -1)], 13: [(13, -1)], 14: [(0, -1), (14, -1)], 15: [(15, -1), (16, -1)], 16: [(16, -1)], 17: [(17, -1), (18, -3), (19, -3)], 18: [(14, -2), (18, -1)], 19: [(19, -1), (19, -3)]}
Causal process: {0: [(0, -1)], 1: [(1, -1)], 2: [(2, -1)], 3: [(3, -1)], 4: [(4, -1), (2, -3)], 5: [(5, -1)], 6: [(6, -1), (19, -2)], 7: [(7, -1)], 8: [(8, -1), (2, -2)], 9: [(9, -1)], 10: [(10, -1)], 11: [(11, -1)], 12: [(12, -1)], 13: [(13, -1)], 14: [(14, -1), (0, -3)], 15: [(15, -1)], 16: [(16, -1)], 17: [(17, -1), (18, -3)], 18: [(18, -1)], 19: [(19, -1)]}
Precision: 0.88
Recall: 0.5238095238095238
F1: 0.6567164179104478
